# import

In [3]:
import sys
sys.path.append('../')
from my_library.library import *
from hyperopt import hp, tpe, Trials, fmin,STATUS_OK
from my_library.funcs import *
from scipy import fftpack
import numpy as np
import matplotlib.pyplot as plt



# path win

In [4]:
path_tpx = '/Users/Owner/Desktop/StockPriceData/Stock_index/TOPIX_10years.csv'
path_225 = '/Users/Owner/Desktop/StockPriceData/Stock_index/NK225_10years.csv'
path_daw = '/Users/Owner/Desktop/StockPriceData/Stock_index/DAW_10years.csv'
path_bear = '/Users/Owner/Desktop/StockPriceData/Stock_index/R225BEAR_10years.csv'
path_tpx_sim = '/Users/Owner/Desktop/StockPriceData/TOPIX/TOPIX_20211208.csv'
path_daw_sim = '/Users/Owner/Desktop/StockPriceData/DAW/DAW_20211208.csv'
save_pickle_path = '/Users/Owner/Desktop/program/Sotsuron/code/wave_pickles'

# funcs

In [5]:
def get_gyosyu_df(path_gyosyu = '/Users/Owner/Desktop/StockPriceData/Gyosyu/'):
    FILE = glob.glob(path_gyosyu+'*.csv')
    df_dict = {}
    for file in FILE:
        name = file.replace(path_gyosyu,'')[:-4]
        df = pd.read_csv(file)
        df = df.rename(columns={df.columns[0]:'nan',df.columns[1]:'nan',df.columns[2]:'nan',\
                                    df.columns[3]:'day',df.columns[4]:'nan',df.columns[5]:'open',\
                                    df.columns[6]:'high',df.columns[7]:'low',df.columns[8]:'close',\
                                        df.columns[9]:'volume',})
        df = df.drop('nan',axis=1)
        df = df.drop(df.index[0])
        df['day'] = pd.to_datetime(df['day'],format='%Y/%m/%d')
        df.set_index('day',inplace=True)
        df_dict[name] = df

    return df_dict,FILE

def return_corr(df):
    x = df['daw_close'].values
    y = df['close'].values
    corr = np.corrcoef(x,y)
    return corr

def split_df(df,start_year,end_year,start_month=1,end_month=12):
    df = df[df.index.year<=end_year]
    df = df[df.index.year>=start_year]
    df = df[df.index.month<=end_month]
    if start_month>0:
        df = df[df.index.month>=start_month]
    return df

def make_value_list(lx,start_year,end_year,path_tpx,path_daw,alpha=0.34,width=20,stride=10,start_month=1,end_month=12):

    lc_dummy = LearnClustering(width=width)
    df_con = lc_dummy.make_df_con(path_tpx,path_daw)
    
    df_con = split_df(df_con,start_year,end_year,start_month,end_month)
    
    x_,z_ = lc_dummy.make_x_data(df_con['close'],stride=stride,test_rate=1.0,width=width)
    length = len(z_)
    value_list = []

    for i in range(length):
        for strategy in ['normal','reverse']:
            try:
                # 本当はXGBSimulation2
                xl = XGBSimulation2(lx,alpha=alpha)
                xl.simulate(path_tpx,path_daw,strategy=strategy,is_validate=True,start_year=start_year,end_year=end_year,df_=z_[i])
                
                trade_log =  xl.trade_log
                total_profit = trade_log['total_profit'].values[0]
                stock_wave = z_[i]
                vt = ValueTable(strategy,alpha,total_profit,trade_log,stock_wave)
                value_list.append(vt)
                
            except Exception as e:
                print(e)
                continue

    return value_list

def return_clx(Value_list):
    Value_good = sorted(Value_list,key=lambda x :x[2],reverse=True)
    Value_bad = sorted(Value_list,key=lambda x :x[2],reverse=False)
    ng = []
    rg = []
    nb = []
    rb = []
    
    # 1sigam = 外れ値 として処理する
    prf_list=[]
    for vg in Value_good:
        total_profit = vg.total_profit
        prf_list.append(total_profit)      
    prf_array = np.array(prf_list)
    st_prf = standarize(prf_array)

    for idx,v in enumerate(Value_good):
        if v.total_profit<=0:break
        # if np.abs(st_prf[idx]) >=1:continue    

        df = v.stock_wave
        strategy = v.strategy
        # print(df)
        # break
        if strategy=="normal":
            ng.append(standarize(df))
        else:
            rg.append(standarize(df))

    prf_list=[]
    for vb in Value_bad:
        total_profit = vb.total_profit
        prf_list.append(total_profit)      
    prf_array = np.array(prf_list)
    st_prf = standarize(prf_array)

    for v in Value_bad:
        if v.total_profit>=0 :break
        # if np.abs(st_prf[idx]) >=1:continue  
        
        df = v.stock_wave
        strategy = v.strategy

        if strategy=="normal":
            nb.append(standarize(df))
        else:
            rb.append(standarize(df))

    x_ng = make_easy_x(ng)
    x_nb = make_easy_x(nb)
    x_rg = make_easy_x(rg)
    x_rb = make_easy_x(rb)
    return x_ng,x_nb,x_rg,x_rb

def return_ffs(lx,x_ng,x_nb,x_rg,x_rb,FFT_simulation,width=20,stride=10,window_type='none',is_high_pass=False,is_low_pass=True,cut_off=3):


    log_dict = {}
    cs_dict = {}
    ffs_dict = {}
    

    random_state=0

    alpha = 0.33
    n_cluster = 1
        
    Fstrategies = []
    Wstrategies = []
    Cstrategies = []
    F_list = []
    Phases = []
    lc_rg = LearnClustering(n_cluster=n_cluster,random_state=random_state)
    lc_rg.learn_clustering3(x_rg,width=width)
    lc_rb = LearnClustering(n_cluster=n_cluster,random_state=random_state)
    lc_rb.learn_clustering3(x_rb,width=width)
    lc_ng = LearnClustering(n_cluster=n_cluster,random_state=random_state)
    lc_ng.learn_clustering3(x_ng,width=width)
    lc_nb = LearnClustering(n_cluster=n_cluster,random_state=random_state)
    lc_nb.learn_clustering3(x_nb,width=width)

    strategy_list = ['normal','stay','reverse','stay']

    j=0
    fft_dummy = FFTSimulation(lx,None,width=width,window_type=window_type,is_low_pass=is_low_pass,is_high_pass=is_high_pass,cut_off=cut_off)
    for lc in [lc_ng,lc_nb,lc_rg,lc_rb]:
        
        for _,key in enumerate(lc.wave_dict):
            wave = lc.wave_dict[key]
            # このタイミングでハイパスフィルタかける
            # make_spectrum 内の関数をいじる
            spe = fft_dummy.make_spectrum(wave)
            ceps = fft_dummy.make_cepstrum(spe)
            F,Amp = fft_dummy.do_fft(wave)
            F = F[:len(F)//2]
            phase = np.degrees(np.angle(F))
            strategy = strategy_list[j]
            fs  = Fstrategy(strategy,alpha,spe)
            ws = Fstrategy(strategy,alpha,wave)
            ph = Fstrategy(strategy,alpha,phase)
            cs = Fstrategy(strategy,alpha,ceps)
            
            Fstrategies.append(fs)
            Wstrategies.append(ws)
            Cstrategies.append(cs)
            Phases.append(ph)
            F_list.append(F)
        j+=1

    return Fstrategies,Phases,F_list,Cstrategies

def return_fft_list(lx,x_,FFT_obj,width=20):

    fft_list = []
    fft_dummy = FFT_obj(lx,None,width=width)
    for wave in x_:
        spe = fft_dummy.make_spectrum(wave)
        fft_list.append(spe)
        
    return fft_list        

def return_cumulative_fst(Fstrategies_new,Fstrategies_old):
    Fstrategies = []
    strategy_list = ['normal','stay','reverse','stay']

    for idx,fst in enumerate(Fstrategies_new):
        spe_new = fst.spectrum
        fst_old = Fstrategies_old[idx]
        spe_old = fst_old.spectrum
        spe_new = standarize(spe_new+spe_old)

        
        fs  = Fstrategy(fst_old.strategy,0.33,spe_new)
        Fstrategies.append(fs)
    
    return Fstrategies

def concat_list(key_list,value_dict):
    value_list = []
    for key in key_list:
        v_lis = value_dict[key]
        for v in v_lis:
            value_list.append(v)
    return value_list

def return_ffs_dict(value_dict,lx_dict,width=40,stride=5,window_type='bla',is_high_pass=False,is_low_pass=True,is_ceps=False,cut_off=3):
    ffs_dict = {}
    trade_dict = {}
    lx_dummy = LearnXGB(num_class=3)
    x_train,y_train,_,_= lx_dummy.make_xgb_data(path_tpx,path_daw,test_rate=1.0)
    _,_ ,x_test,y_test = lx_dummy.make_xgb_data(path_tpx,path_daw,test_rate=0.9)
    F_list = []
    F_lis_dict = {}
    limit_year = 2009

    for year in range(limit_year+2,2022):
        print(year)

        start_month=1
        end_month = 12
        start_year = year
        end_year = year
        value_list = value_dict[str(year-1)]
        lx_ = lx_dict[str(year-1)]

        x_ng,x_nb,x_rg,x_rb = return_clx(value_list)

        try:
            Fstrategies,_,F_list,Cstrategies = return_ffs(lx_,x_ng,x_nb,x_rg,x_rb,FFTSimulation,width=width,stride=stride,\
                window_type=window_type,is_high_pass=is_high_pass,is_low_pass=is_low_pass,cut_off=cut_off)
            """
            # Fstrategies をどんどん加算していく    
            # if len(ffs_dict)>0:
            #     last_key = next(reversed(ffs_dict),None)
            #     Fstrategies_old = ffs_dict[last_key].Fstrategies
            #     Fstrategies = return_cumulative_fst(Fstrategies,Fstrategies_old)
            """
            
        except Exception as e:
            print(e)
            last_key = next(reversed(ffs_dict),None)
            Fstrategies = ffs_dict[last_key].Fstrategies
            # Cepstrum で　やってみる
            # Cstrategies = ffs_dict[last_key].Fstrategies
            
        
        # ffs = FFTSimulation(lx,Fstrategies,width=width,window_type=window_type)
        ffs = FFTSimulation(lx_,Fstrategies,width=width,window_type=window_type,is_ceps=is_ceps,\
            is_high_pass=is_high_pass,is_low_pass=is_low_pass,cut_off=cut_off)
        ffs.simulate(path_tpx,path_daw,start_year=year,end_year=year,is_validate=True)
        
        ffs_dict[str(year)] = ffs
        trade_dict[str(year)] = ffs.trade_log
        F_lis_dict[str(year)] = F_list
    
    return ffs_dict, trade_dict, F_lis_dict


# load_data

In [6]:
value_dict = load_pickle(save_pickle_path+'/value_dict.pickle')
lx_dict = load_pickle(save_pickle_path+'/lx_dict.pickle')

# save_data

In [ ]:
"""
save_path = '/Users/Owner/Desktop/program/Sotsuron/code/wave_pickles/value_dict.pickle'
save_pickle(save_path,value_dict)

save_path = '/Users/Owner/Desktop/program/Sotsuron/code/wave_pickles/lx_dict.pickle'
save_pickle(save_path,lx_dict)
"""

# 周波数成分プロット

In [ ]:
plt.clf()
for idx,key in enumerate(ffs_dict):
    Fstrategies = ffs_dict[key].Fstrategies
    # 0 : ng, 1 : nb, 2 : rg, 3: rb
    print(key)
    plt.clf()
    fig, ax = plt.subplots(figsize=(10, 6), facecolor="w")
    
    for idx,fs in enumerate(Fstrategies):
        if idx==0:
            label='ng'
        elif idx == 1:
            label='nb'
        elif idx == 2:
            label='rg'
        elif idx == 3:
            label='rb'
        ax.plot(fs.spectrum,label=label,marker='o',markersize=7)
    

    ax.grid()
    ax.legend()
    plt.show()

# 収益確認

In [ ]:
total_profit = 0
mean_profit = 0
mean_profit_rate = 0
total_profit_rate = 0
for idx,key in enumerate(trade_dict):
    print(key)
    print(trade_dict[key])
    td = trade_dict[key]
    total_profit += td['total_profit'].values[0]
    total_profit_rate += td['profit rate'].values[0]
    
mean_profit = total_profit/(idx+1)
mean_profit_rate = (total_profit_rate)/(idx+1)

print('total profit :{0}'.format(total_profit))
print('mean profit :{0}'.format(mean_profit))
print('total profit rate :{0}'.format(total_profit_rate))
print('mean profit rate : {0}'.format(mean_profit_rate))

# Low pass vs High pass